# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 9 2022 11:31AM,252719,10,Enova-11074,Emerginnova,Released
1,Dec 9 2022 11:27AM,252718,10,PRF-44162,Bio-PRF,Released
2,Dec 9 2022 11:27AM,252718,10,PRF-44174,Bio-PRF,Released
3,Dec 9 2022 11:27AM,252718,10,PRF-44177,Bio-PRF,Released
4,Dec 9 2022 11:27AM,252718,10,PRF-44180,Bio-PRF,Released
5,Dec 9 2022 11:27AM,252718,10,PRF-44183,Bio-PRF,Released
6,Dec 9 2022 11:27AM,252718,10,PRF-44188,Bio-PRF,Released
7,Dec 9 2022 11:27AM,252718,10,PRF-44191,Bio-PRF,Released
8,Dec 9 2022 11:27AM,252718,10,PRF-44194,Bio-PRF,Released
9,Dec 9 2022 11:27AM,252718,10,PRF-44200,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,252710,Released,1
46,252714,Released,3
47,252715,Released,1
48,252718,Released,11
49,252719,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252710,NaN,NaN,1.0
252714,NaN,NaN,3.0
252715,NaN,NaN,1.0
252718,NaN,NaN,11.0
252719,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252619,0.0,0.0,2.0
252624,0.0,0.0,1.0
252625,0.0,0.0,4.0
252627,0.0,1.0,2.0
252636,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252619,0,0,2
252624,0,0,1
252625,0,0,4
252627,0,1,2
252636,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252619,0,0,2
1,252624,0,0,1
2,252625,0,0,4
3,252627,0,1,2
4,252636,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252619,,,2
1,252624,,,1
2,252625,,,4
3,252627,,1,2
4,252636,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 9 2022 11:31AM,252719,10,Emerginnova
1,Dec 9 2022 11:27AM,252718,10,Bio-PRF
12,Dec 9 2022 10:56AM,252715,18,Innova Softgel LLC
13,Dec 9 2022 10:34AM,252714,10,ISDIN Corporation
16,Dec 9 2022 10:13AM,252710,21,"NBTY Global, Inc."
17,Dec 9 2022 10:04AM,252708,21,"NBTY Global, Inc."
18,Dec 9 2022 10:01AM,252707,21,"NBTY Global, Inc."
19,Dec 9 2022 9:56AM,252706,21,"NBTY Global, Inc."
20,Dec 9 2022 9:08AM,252703,10,Emerginnova
21,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 9 2022 11:31AM,252719,10,Emerginnova,,,1
1,Dec 9 2022 11:27AM,252718,10,Bio-PRF,,,11
2,Dec 9 2022 10:56AM,252715,18,Innova Softgel LLC,,,1
3,Dec 9 2022 10:34AM,252714,10,ISDIN Corporation,,,3
4,Dec 9 2022 10:13AM,252710,21,"NBTY Global, Inc.",,,1
5,Dec 9 2022 10:04AM,252708,21,"NBTY Global, Inc.",,,1
6,Dec 9 2022 10:01AM,252707,21,"NBTY Global, Inc.",,,1
7,Dec 9 2022 9:56AM,252706,21,"NBTY Global, Inc.",,,1
8,Dec 9 2022 9:08AM,252703,10,Emerginnova,,,1
9,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 11:31AM,252719,10,Emerginnova,1,,
1,Dec 9 2022 11:27AM,252718,10,Bio-PRF,11,,
2,Dec 9 2022 10:56AM,252715,18,Innova Softgel LLC,1,,
3,Dec 9 2022 10:34AM,252714,10,ISDIN Corporation,3,,
4,Dec 9 2022 10:13AM,252710,21,"NBTY Global, Inc.",1,,
5,Dec 9 2022 10:04AM,252708,21,"NBTY Global, Inc.",1,,
6,Dec 9 2022 10:01AM,252707,21,"NBTY Global, Inc.",1,,
7,Dec 9 2022 9:56AM,252706,21,"NBTY Global, Inc.",1,,
8,Dec 9 2022 9:08AM,252703,10,Emerginnova,1,,
9,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 11:31AM,252719,10,Emerginnova,1,,
1,Dec 9 2022 11:27AM,252718,10,Bio-PRF,11,,
2,Dec 9 2022 10:56AM,252715,18,Innova Softgel LLC,1,,
3,Dec 9 2022 10:34AM,252714,10,ISDIN Corporation,3,,
4,Dec 9 2022 10:13AM,252710,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 11:31AM,252719,10,Emerginnova,1.0,NaN,NaN
1,Dec 9 2022 11:27AM,252718,10,Bio-PRF,11.0,NaN,NaN
2,Dec 9 2022 10:56AM,252715,18,Innova Softgel LLC,1.0,NaN,NaN
3,Dec 9 2022 10:34AM,252714,10,ISDIN Corporation,3.0,NaN,NaN
4,Dec 9 2022 10:13AM,252710,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 11:31AM,252719,10,Emerginnova,1.0,0.0,0.0
1,Dec 9 2022 11:27AM,252718,10,Bio-PRF,11.0,0.0,0.0
2,Dec 9 2022 10:56AM,252715,18,Innova Softgel LLC,1.0,0.0,0.0
3,Dec 9 2022 10:34AM,252714,10,ISDIN Corporation,3.0,0.0,0.0
4,Dec 9 2022 10:13AM,252710,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4548136,119.0,3.0,1.0
15,505371,7.0,0.0,0.0
18,252715,1.0,0.0,0.0
19,1263410,6.0,0.0,0.0
20,252653,0.0,1.0,0.0
21,5053263,20.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4548136,119.0,3.0,1.0
1,15,505371,7.0,0.0,0.0
2,18,252715,1.0,0.0,0.0
3,19,1263410,6.0,0.0,0.0
4,20,252653,0.0,1.0,0.0
5,21,5053263,20.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119.0,3.0,1.0
1,15,7.0,0.0,0.0
2,18,1.0,0.0,0.0
3,19,6.0,0.0,0.0
4,20,0.0,1.0,0.0
5,21,20.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,15,Released,7.0
2,18,Released,1.0
3,19,Released,6.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,0.0,0.0,0.0,1.0,0.0
Released,119.0,7.0,1.0,6.0,0.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,1.0,0.0
2,Released,119.0,7.0,1.0,6.0,0.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,1.0,0.0
2,Released,119.0,7.0,1.0,6.0,0.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()